In [ ]:
##Packages
from sportsreference.ncaab.schedule import Schedule
from sportsreference.ncaab.teams import Teams
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import pandas
import numpy
import warnings

warnings.filterwarnings('ignore')

pandas.set_option('display.max_columns', 500)
pandas.set_option('display.max_rows', 500)
data = pandas.read_csv('ncaab_box_scores.csv')
data = data[data.columns[-47:]]
data['ranked'] = numpy.where(data['team_rank']>0, 1, 0)
data['ranked_opp'] = numpy.where(data['opp_rank']>0, 1, 0)

log_reg_drop = ['team','unique_game','opponent','won','game_date','team_rank','opp_rank']

# FIELDS_TO_DROP = ['team','unique_game','opponent','won','game_date','team_assist_pct',
# 'team_block_pct','team_def_reb_pct','team_ft_rate','team_ft_pct','team_off_reb_pct','team_rank','team_stl_pct','team_3pt_rate',
# 'team_to_pct','team_2pt_rate','opp_assist_pct','opp_block_pct','opp_def_reb_pct','opp_ft_rate','opp_ft_pct','opp_off_reb_pct',
# 'opp_rank','opp_stl_pct','opp_3pt_rate','opp_to_pct','opp_2pt_rate','team_fg_pct','opp_fg_pct','team_reb_pct',
# 'opp_reb_pct','team_3pt_pct','team_2pt_pct','opp_3pt_pct','opp_2pt_pct']

data = pandas.get_dummies(data,prefix=['team_home_away'], columns=['team_home_away'])

In [26]:
baylor_home_data = pandas.DataFrame(data[numpy.logical_and(numpy.logical_and(data['team']=='BAYLOR',data['team_home_away_Home']==1),
                                          data['opp_rank']>0)].mean())
wv_away_data = pandas.DataFrame(data[numpy.logical_and(numpy.logical_and(data['team']=='WEST-VIRGINIA',data['team_home_away_Home']==0),
                                          data['opp_rank']>0)].mean())
# matchup = pandas.concat([baylor_home_data,wv_away_data],axis=0,sort=True)    
# matchup
## NEED TO MAKE A DATAFRAME WITH BAYLOR TEAM FIELDS AND WV OPP FIELDS TO PREDICT BAYLOR OUTCOME ##

,0
team_assist_pct,0.517667
team_block_pct,0.187333
team_def_rating,94.466667
team_def_reb_pct,0.717667
team_eff_fg_pct,0.458667
team_fg_pct,0.393000
team_ft_rate,0.329333
team_ft_pct,0.684667
team_off_rating,101.866667
team_off_reb_pct,0.327333


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn import metrics

x = data.drop(log_reg_drop, 1).dropna().drop_duplicates()
y = data['won']

scaler = preprocessing.StandardScaler()
x_scaled = scaler.fit_transform(x)

x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=.3, random_state=42)

logR = LogisticRegression()
logR.fit(x_train, y_train)

predictions_logr = logR.predict(x_test)
prediction_strength_logr = logR.predict_proba(x_test)
score_logr = logR.score(x_test, y_test)

cm_logr = metrics.confusion_matrix(y_test, predictions_logr, labels = [0,1])
cmtx_logr = pandas.DataFrame(
    cm_logr, 
    index=['true:no', 'true:yes'], 
    columns=['pred:no', 'pred:yes'])
print(cmtx_logr)

In [ ]:
## Figures out most important features to use
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE,RFECV

rfecv = RFECV(estimator=logR, step=1, scoring='accuracy')
rfecv.fit(x_scaled, y)

plt.figure(figsize=(16, 9))
plt.title('Recursive Feature Elimination with Cross-Validation', fontsize=18, fontweight='bold', pad=20)
plt.xlabel('Number of features selected', fontsize=14, labelpad=20)
plt.ylabel('% Correct Classification', fontsize=14, labelpad=20)
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_, color='#303F9F', linewidth=3)

rankings = pandas.DataFrame()
rankings['attribute'] = x.columns
selector = RFE(logR,n_features_to_select=1)
selector = selector.fit(x_scaled,y)
rankings['rank'] = selector.ranking_

plt.show()
print(rankings.sort_values('rank'))

In [ ]:
result_x = sport_ref_data.iloc[:,[35,12,33,4,11,42,2]]

result_x_scaled = scaler.fit_transform(result_x)

predictions_results = logR.predict(result_x_scaled)
prediction_strength_results = logR.predict_proba(result_x_scaled)
sport_ref_data['prediction'] = predictions_results
sport_ref_data['confidence'] = prediction_strength_results[:,1]
preds_current_season_append = sport_ref_data

preds_current_season_append.sort_values('confidence',ascending=False).head(75).iloc[:,[35,12,33,4,11,42,2,46]]

In [ ]:
# X = data.drop(FIELDS_TO_DROP, 1).dropna().drop_duplicates()
# y = data[['team_pts', 'opp_pts']].values
# X_train, X_test, y_train, y_test = train_test_split(X, y)

# parameters = {'bootstrap': False,
#               'min_samples_leaf': 3,
#               'n_estimators': 50,
#               'min_samples_split': 10,
#               'max_features': 'sqrt',
#               'max_depth': 6}
# model = RandomForestRegressor(**parameters)
# model.fit(X_train, y_train)

# predictions = model.predict(X_test)
# errors = abs(predictions - y_test)
# print('Mean Absolute Error:', round(numpy.mean(errors), 2), 'points.')


In [ ]:
# importances = list(model.feature_importances_)
# feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(X, importances)]
# feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# [print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

In [ ]:
# data2 = data
# data2['pred_team_pts'] = pandas.DataFrame(predictions)[0]
# data2['pred_opp_pts'] = pandas.DataFrame(predictions)[1]

# accuracy = abs(data2['pred_team_pts'] - data2['team_pts'])
# accuracy.dropna().mean()